# Locally interact with remote objects 
We can use a notebook to interactively work with the remote Training. 

In [ ]:
import kubetorch as kt 
num_nodes = 3

img = kt.images.dask().pip_install(
    [
        "gcsfs", # For Google Cloud Storage, change for your cloud provider
        "lightgbm",
    ]
)

compute = kt.compute(cpus = "8", memory = "32", image=img).up_if_not()

In [ ]:
from lightgbm_dask import LightGBMModelTrainer
trainer = kt.cls(LightGBMModelTrainer).to(compute).distribute('dask', num_nodes = num_nodes) 

In [ ]:
dataset_path = "gs://rh-demo-external/*.parquet"  # 2024 NYC Taxi Data
X_vars = ["passenger_count", "trip_distance", "fare_amount"]
y_var = "tip_amount"


trainer.load_client()
trainer.load_data(dataset_path)

In [ ]:
new_date_columns = trainer.preprocess(date_column="tpep_pickup_datetime")
X_vars = X_vars + new_date_columns
trainer.train_test_split(target_var=y_var, features=X_vars)


In [ ]:
# Train, test, and save the model
trainer.train_model()


In [ ]:
# Run an inference on the remote model 
import numpy as np 
import dask.array as da
input_data = [3, 5, 110, 5, 11, 3, 6]

trainer.predict(input_data)

INFO | 2024-11-12 22:35:13 | runhouse.servers.http.http_client:439 | Calling my_trainer.predict


INFO:httpx:HTTP Request: GET http://127.0.0.1:32302/logs/my_trainer/my_trainer_predict@20241112@223513_267805/pickle "HTTP/1.1 200 OK"


------------
py-2-new-gcp
------------
dask.array<_predict_part, shape=(1,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
[10.87070356]
INFO | 2024-11-12 22:35:13 | runhouse.servers.http.http_client:504 | Time to call my_trainer.predict: 0.58 seconds


10.870703558582909